In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, LlamaTokenizer
import torch
import json
import os

In [2]:
tokenizer = LlamaTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/')
batch_size = 4
input_ids = tokenizer.encode('os:', return_tensors='pt')
# tokenizer(
#     # batch_size*[''],
#     ['4235 ', '5462 ', '7132 ', '6460 '], 
#     return_tensors="pt"
# ).input_ids  # Batch size 1

def sample(ckpt_dir, step):
    output_dir = os.path.join(ckpt_dir, f'checkpoint-{step}') #f'/mnt/data/sonia/ckpts/llama-2-7b-jul25/checkpoint-{step}/'
    print(output_dir)
    model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
    model = model.merge_and_unload() #returns type transformers.models.llama.modeling_llama.LlamaForCausalLM
    
    outputs = model.generate(input_ids, max_new_tokens=100).cpu()
    out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    model = model.cpu()
    return out

In [10]:
s = sample('/mnt/data/sonia/ckpts/debug', 20)
s

/mnt/data/sonia/ckpts/debug/checkpoint-20


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['os:\n\n    # 1. 설정 파일 생성\n    # 2. 설정 파일 읽어오기\n    # 3. 설정 파일 쓰기\n    # 4. 설정 파일 삭제\n    # 5. 설정 �']

In [9]:
print(s[0])

os:
1 {'os': 'Windows Server 2012 R2 Datacenter 9600', 'ip_str': '10.10.10.10', 'port': 445, 'module': 'smb', 'cpe': None, 'cpe_count': 0, 'category': 'file_sharing', 'os_generic': 'windows server', 'single_cpe': None}


In [6]:
tokenizer = LlamaTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/')
batch_size = 4
input = tokenizer(
    # batch_size*[''],
    ['There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.\n',], 
    return_tensors="pt",
)  # Batch size 1
input = {x:input[x].cuda() for x in input}

In [7]:
step=60
output_dir = f'/mnt/data/sonia/ckpts/sent3/checkpoint-{step}/'
print(output_dir)
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
# model=model.merge_and_unload()

/mnt/data/sonia/ckpts/sent3/checkpoint-60/


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
outputs = model.generate(**input, max_new_tokens=58)
out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(out[0], len(out[0]), len(out[0])-5, sep='\n')

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.
There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.
263
258


max_length

In [19]:
for i in range(1,len(outputs[0])):
    print(tokenizer.batch_decode(outputs[:, 0:i]))

['<s>']
['<s>There']
['<s>There is']
['<s>There is a']
['<s>There is a Ubuntu']
['<s>There is a Ubuntu server']
['<s>There is a Ubuntu server visible']
['<s>There is a Ubuntu server visible at']
['<s>There is a Ubuntu server visible at IP']
['<s>There is a Ubuntu server visible at IP ']
['<s>There is a Ubuntu server visible at IP 4']
['<s>There is a Ubuntu server visible at IP 43']
['<s>There is a Ubuntu server visible at IP 43.']
['<s>There is a Ubuntu server visible at IP 43.2']
['<s>There is a Ubuntu server visible at IP 43.20']
['<s>There is a Ubuntu server visible at IP 43.205']
['<s>There is a Ubuntu server visible at IP 43.205.']
['<s>There is a Ubuntu server visible at IP 43.205.1']
['<s>There is a Ubuntu server visible at IP 43.205.13']
['<s>There is a Ubuntu server visible at IP 43.205.13.']
['<s>There is a Ubuntu server visible at IP 43.205.13.2']
['<s>There is a Ubuntu server visible at IP 43.205.13.24']
['<s>There is a Ubuntu server visible at IP 43.205.13.243']
['<s>There

In [9]:
outputs

tensor([[    1,  1670,   338,   263,  8294,  1923,  7962,   472,  5641, 29871,
         29946, 29941, 29889, 29906, 29900, 29945, 29889, 29896, 29941, 29889,
         29906, 29946, 29941, 29892,  2011, 29871, 29906, 29906, 29892, 27032,
           278,  2669,   274,   412,  8419, 29874, 29901,  3150, 29890,  4928,
         29901, 22156,   845, 29901, 29947, 29889, 29906, 29886, 29896,  8294,
         29899, 29946,  8767, 29900, 29889, 29945, 29889,    13,  8439,   338,
           263,  8294,  1923,  7962,   472,  5641, 29871, 29946, 29941, 29889,
         29906, 29900, 29945, 29889, 29896, 29941, 29889, 29906, 29946, 29941,
         29892,  2011, 29871, 29906, 29906, 29892, 27032,   278,  2669,   274,
           412,  8419, 29874, 29901,  3150, 29890,  4928, 29901, 22156,   845,
         29901, 29947, 29889, 29906, 29886, 29896,  8294, 29899, 29946,  8767,
         29900, 29889, 29945, 29889,     2]], device='cuda:0')

In [20]:
tokenizer.batch_decode(outputs[0][10:13]), outputs[0][10:13]

(['4', '3', '.'], tensor([29946, 29941, 29889], device='cuda:0'))

In [21]:
tokenizer.batch_decode([29941, 29942, 29943, 29944, 29945, 29946])

['3', 'в', 'F', 'л', '5', '4']